<a href="https://colab.research.google.com/github/jus-tinian/colab_shares/blob/master/amzn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

In [0]:
def get_soup(link):

  page = requests.get(link)
  soup = BeautifulSoup(page.text, 'html.parser')

  return soup

In [0]:
def get_tags(soupy_tags):

  tags = list()

  for tag in soupy_tags.find_all():
    tags.append(tag.name)

  tags = list(set(tags))

  return tags

In [0]:
def make_link(linky_text):
  return f'<a target="_blank" href="{linky_text}">Link</a>'

In [0]:
def find_xml(soup_field):

  xml = ''.join(['https://www.sec.gov' + i.attrs['href'] for i in soup_field.find_all('a') if '_htm.xml' in i.text])

  if xml == '':
    xml = ''.join(['https://www.sec.gov' + i.find_next('a').attrs['href'] for i in soup_field.find_all('td') if i.text == 'XBRL INSTANCE DOCUMENT'])

  return xml

In [0]:
# getting 10Ks
historical = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001018724&type=10-K&dateb=&owner=exclude&output=xml&count=100'

In [0]:
historical = get_soup(historical)

In [8]:
company = [i.text for i in historical.find_all('name')]
company

['AMAZON COM INC']

In [0]:
hrefs = [i.text for i in historical.find_all('filinghref')]

In [0]:
dates = [i.text for i in historical.find_all('datefiled')]

In [0]:
filing = [i.text for i in historical.find_all('type')]

In [0]:
desc = [i.text for i in historical.find_all('sicdescription')]

In [0]:
data = pd.DataFrame({'Type': filing, 'Date': dates, 'Full_Report': hrefs})

In [0]:
data = data.loc[data.Type == '10-K', :].iloc[:5]

In [0]:
data['XML'] = data.Full_Report.copy().apply(get_soup).apply(find_xml)

In [0]:
dataMod = data.copy()

In [0]:
dataMod['Full_Report'] = dataMod['Full_Report'].apply(make_link)

In [0]:
dataMod['XML'] = dataMod['XML'].apply(make_link)

In [0]:
dataMod = dataMod.head()

In [20]:
display(HTML(dataMod.to_html(escape=False)))

,Type,Date,Full_Report,XML
0,10-K,2020-01-31,Link,Link
1,10-K,2019-02-01,Link,Link
2,10-K,2018-02-02,Link,Link
3,10-K,2017-02-10,Link,Link
4,10-K,2016-01-29,Link,Link


In [0]:
dataMod['soup'] = data.XML.apply(get_soup)

In [27]:
list(set([i.name for i in dataMod.soup.iloc[0].find_all()]))

['us-gaap:deferredincometaxexpensebenefit',
 'us-gaap:othercommitmentdueinfourthyear',
 'us-gaap:incometaxreconciliationtaxcredits',
 'us-gaap:deferredtaxassetstaxdeferredexpensecompensationandbenefitssharebasedcompensationcost',
 'us-gaap:netincomeloss',
 'us-gaap:compensationrelatedcostspolicytextblock',
 'us-gaap:financeleaserightofuseassetamortization',
 'us-gaap:weightedaveragenumberofdilutedsharesoutstanding',
 'us-gaap:assets',
 'amzn:sharebasedcompensationarrangementbysharebasedpaymentawardequityinstrumentsotherthanoptionsvestingnexttwelvemonths',
 'dei:entityaddresspostalzipcode',
 'amzn:financingobligationsfutureminimumpaymentsdueinthreeyears',
 'us-gaap:netcashprovidedbyusedinfinancingactivities',
 'amzn:financeleaserightofuseassetaccumulatedamortization',
 'us-gaap:longtermdebtcurrent',
 'us-gaap:employeeservicesharebasedcompensationtaxbenefitfromcompensationexpense',
 'us-gaap:capitalleasedassetsgross',
 'us-gaap:othercomprehensiveincomelossforeigncurrencytransactionandtra